In [2]:
from IPython.display import display, HTML, Image, clear_output
import time
import random
import ipywidgets as widgets
from jupyter_ui_poll import ui_events
import requests
from bs4 import BeautifulSoup
import json
from ipywidgets import FloatProgress

In [3]:
# These codes came from the teaching material provided in the BIOS0030 module 
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True): 
    """
    Waits for an event with optional timeout.

    Parameters:
    - timeout: Timeout value in seconds. Default is -1 (no timeout).
    - interval: Interval in seconds to check for events. Default is 0.001.
    - max_rate: Maximum rate of event polling per second. Default is 20.
    - allow_interrupt: Boolean flag to allow interruption. Default is True.

    Returns:
    - event_info: A dictionary containing information about the event.
      - 'type': Type of the event.
      - 'description': Description of the event.
      - 'time': Time of occurrence of the event.
    """
    start_wait = time.time()    # Initialize time for timeout
    event_info['type'] = ""      #Type of the event
    event_info['description'] = ""    # Description of the event
    event_info['time'] = -1           # Time of occurence of the event
    n_proc = int(max_rate*interval)+1
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            ui_poll(n_proc)
            if(timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False

            if allow_interupt==True and event_info['description'] != "":
                keep_looping = False
            time.sleep(interval)
    return event_info

def register_event(btn):
     """
    Registers a click event for the given button.

    Parameters:
    - btn: The button object for which the click event is registered.

    Returns:
    - None
    """
     event_info['type'] = "click"
     event_info['description'] = btn.description
     event_info['time'] = time.time()    #Time of the click event
     return

In [4]:
# These codes came from the teaching material provided in the BIOS0030 module 
def data_consent():
     """
    Presents a data consent form to the user.

    Parameters:
    -None

    Returns:
    - None
    """
     data_consent_info = """DATA CONSENT INFORMATION:

    Please read:
    
    We wish to record your response data
    to an anonymised public data repository. 
    Your data will be used for educational teaching purposes
    practising data analysis and visualisation.
    
    Please type   yes   in the box below if you consent to the upload."""
    
     print(data_consent_info)
     result = input("> ") 
     result = result.lower()
    
     if result == "yes": 
         print("Thanks for your participation.")
         print("Please contact a.fedorec@ucl.ac.uk")
         print("If you have any questions or concerns")
         print("regarding the stored results.")
        
     else: 
         # end code execution by raising an exception
         raise(Exception("User did not consent to continue test."))

     return


In [5]:
def answer_options(Option1, Option2, Option3, Option4):
    """
    Displays multiple answer options for the user to choose from.

    Parameters:
    - Option1: Description of the first answer option.
    - Option2: Description of the second answer option.
    - Option3: Description of the third answer option.
    - Option4: Description of the fourth answer option.

    Returns:
    - None
    """
    # Create buttons for each answer option
    btn1 = widgets.Button(description=Option1)
    btn2 = widgets.Button(description=Option2)
    btn3 = widgets.Button(description=Option3)
    btn4 = widgets.Button(description=Option4)

    # Register click events for each button 
    btn1.on_click(register_event)
    btn2.on_click(register_event)
    btn3.on_click(register_event)
    btn4.on_click(register_event)

    # Arrange and display the button horizontally 
    panel = widgets.HBox([btn1, btn2, btn3, btn4])
    display(panel)
    return

In [6]:
# These codes came from the teaching material provided in the BIOS0030 module 
def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [7]:
# Creating a data dictionary for uploading the data to the google form 
def data_dictionary():
    global data_dict
    data_dict={
        'username' : [],
        'age' : [],
        'gender' : [],
        'tiredness' : [],
        'sport_freq' : [],
        'Question1': [],
        'Question2': [], 
        'Question3': [],
        'Question4': [],
        'Question5': [],
        'Question6': [],
        'Question7': [],
        'Question8': [], 
        'Question9': [], 
        'Question10': [],
        'Question11': [],
        'Question12': [],
        'Question13': [],
        'Question14': [],
        'Question15': [],
        'Question16': [],
        'Question17': [],
        'Question18': [],
        'Question19': [],
        'Question20': [],
        'points': [],
        'time_taken': []
        }
    return

In [17]:
def the_starting_page():
    """
    Displays the starting page for the memory test and collects user information.

    Returns:
    - None
    """
    id_instructions = """

    Enter your anonymised ID
    
    To generate an anonymous 4-letter unique user identifier please enter:
    - two letters based on the initials (first and last name) of a childhood friend
    - two letters based on the initials (first and last name) of a favourite actor / actress
    
    e.g. if your friend was called Charlie Brown and film star was Tom Cruise
         then your unique identifier would be CBTC
    """
    
    print(id_instructions)
    username = input("> ")
    print("User entered id:", username)
    data_dict['username'].append(username) # Append name to data dictionary
    print("What is your age?")
    while True:
        age = input("Please enter a number ")
        try:
            val = int(age) # Check if input is an integer
            data_dict['age'].append(age) #Append age to data dictionary
            break; # Exit loop if input is valid
        except ValueError:
            print("This is not a number. Please enter a valid number")
    print("What is your gender")
    gender = input("Enter Male or Female")
    gender = gender.lower()
    data_dict['gender'].append(gender)  #Append gender to data dictionary
    clear_output()
    time.sleep(1)
    tiredness_scale = """

    Based on the scale below, please rate how tired/sleepy you are right now

    1 - Extremely alert
    2 - Very Alert
    3 - Alert
    4 - Rather Alert
    5 - Neither alert nor sleepy
    6 - Some signs of sleepiness
    7 - Sleepy, but no effort to keep awake
    8 - Sleepy, but some effort to keep awake
    9 - Very sleepy, great effort to keep awake, fighting sleep
    10 - Extremely sleepy, can't keep awake

    """
    print(tiredness_scale)
    while True:
        tiredness = input("Please enter a number from 1-10")
        try:
            val=int(tiredness)       
            data_dict['tiredness'].append(tiredness)
            break;
        except ValueError:
            print("Please enter a valid number")
    clear_output()
    
    Sports_scale = """
    Based on the scale below, please indicate how of often do you do sports in a week

    Frequent(>5 days a week)
    Often (3-4 days a week)
    Sometimes (1-2 days a week)
    Never (0 days a week)
    """
    print(Sports_scale)
    answer_options("Frequent", "Often", "Sometimes", "Never")
    result = wait_for_event()
    clear_output()
    sport_freq = result['description']
    data_dict['sport_freq'].append(sport_freq)
    print("Welcome! This is a memory test")
    time.sleep(2)
    print("You have 20 seconds to memorize each image and 10 seconds to answer each question")
    time.sleep(3)
    print("Goodluck")
    time.sleep(3)
    clear_output()
    return

In [10]:
def the_images(picture):        
    """
    Displays an image for 20 seconds to allow the user to memorize it.

    Parameters:
    - picture: Path to the image file.

    Returns:
    - None
    """
    print("You have 20 seconds to memorize this image")
    time.sleep(2)        
    display(Image(picture, width = 450))  # Disply the images
    time.sleep(20)
    clear_output()
    print("Time's up")
    time.sleep(2)
    clear_output()

    return    

In [11]:
def first_set_questions(picture1):
    """
    Presents the first set of questions based on an image.

    Parameters:
    - picture1: Path to the image file.

    Returns:
    - points: Total points earned in the first set of questions.
    """

    global points, progress_bar, count
    # Initialize the progress bar for total answeres answered
    progress_bar = FloatProgress(min=0, max=25, description='Progress:')
    count = 0   # Counter for the total quesions answered 
    time.sleep(1)
    print("Answer the following questions")
    time.sleep(1)
    # Initialize the counter(points) for the total of correct answers  
    points = 0
    print("Question 1: What is the color of the star?")
    time.sleep(1)
    answer_options("Red", "Purple", "Yellow", "Green")
    result = wait_for_event(timeout=10) # Wait for user response for 10 seconds
    clear_output()
    ans1 = result['description']
    data_dict['Question1'].append(ans1)
    if result['description'] == "Red":
        points = points +1 #Increment points if answer is correct
    progress_bar.value +=1 #Increment progress bar
    display(progress_bar)
    count +=1 # Increment question answered
    print(f'Answered: {count}/20')

    # Repeat the above process for subsequent questions
    print("Question 2: What shape is underneath the pentagon?")
    time.sleep(1)
    answer_options("Star", "Circle", "Rectangle", "Square")
    result = wait_for_event(timeout=10)
    clear_output()
    ans2 = result['description']
    data_dict['Question2'].append(ans2)
    if result['description'] == "Rectangle":
        points = points + 1
    progress_bar.value +=1
    display(progress_bar)
    count +=1
    print(f'Answered:{count}/20')

    print("Question 3: Which shape has the color yellow?")
    time.sleep(1)
    answer_options("Triangle", "Square", "Pentagon", "Plus")
    result = wait_for_event(timeout=10)
    clear_output()
    ans3 = result['description']
    data_dict['Question3'].append(ans3)
    if result['description'] == "Triangle":
        points = points + 1
    progress_bar.value +=1
    display(progress_bar)
    count +=1
    print(f'Answered: {count}/20')
    
    print("Question 4: What shape is in between the triangle and the diamond?")
    time.sleep(1)
    answer_options("Diamond", "Circle", "Triangle", "Octagon")
    result = wait_for_event(timeout=10)
    clear_output()
    ans4 = result['description']
    data_dict['Question4'].append(ans4)
    if result['description'] == "Circle":
        points = points + 1
    progress_bar.value +=1
    display(progress_bar)
    count +=1
    print(f'Answered: {count}/20')
    
    print("Question 5: What shape is located at this location?")
    time.sleep(1)
    display(Image(picture1, width = 300))
    answer_options("Square", "Pentagon", "Rectangle", "Star")
    result = wait_for_event(timeout=10)
    clear_output()
    ans5 = result['description']
    data_dict['Question5'].append(ans5)
    if result['description'] == "Star":
        points = points + 1
    print("Next challenge")
    time.sleep(2)
    progress_bar.value +=1
    count +=1
    clear_output()
    return points

In [12]:
def second_set_questions(picture2):
    """
    Presents the second set of questions based on an image.

    Parameters:
    - picture2: Path to the image file.

    Returns:
    - points: Total points earned in the second set of questions.
    """

    global points, count, progress_bar
    display(progress_bar)
    print(f'Answered: {count}/20')
    time.sleep(1)
    print("Answer the following questions")
    time.sleep(1)
    print("Question 6: What is the color of the smiley face?")
    time.sleep(1)
    answer_options("Blue", "Purple", "Red", "Yellow")
    result = wait_for_event(timeout=10)
    clear_output()
    ans6 = result['description']
    data_dict['Question6'].append(ans6)
    if result['description'] == "Purple":
        points = points + 1
    progress_bar.value +=1
    display(progress_bar)
    count +=1
    print(f'Answered: {count}/20')

    print("Question 7:How many arrows are there?")
    time.sleep(1)
    answer_options("1", "0", "3", "2")
    result = wait_for_event(timeout=10)
    clear_output()
    ans7 = result['description']
    data_dict['Question7'].append(ans7)
    if result['description'] == "2":
        points = points + 1
    progress_bar.value +=1
    display(progress_bar)
    count +=1
    print(f'Answered: {count}/20')

    print("Question 8: Which way is the arrow in the higlighted box is facing?")
    time.sleep(1)
    display(Image(picture2, width = 300))
    answer_options("Up", "Down", "Left","Right")
    result = wait_for_event(timeout=10)
    clear_output()
    ans8 = result['description']
    data_dict['Question8'].append(ans8)
    if result['description'] == "Up":
        points = points + 1
    progress_bar.value +=1
    display(progress_bar)
    count +=1
    print(f'Answered: {count}/20')

    print("Question 9: What color is the heart?")
    time.sleep(1)
    answer_options("Green", "Pink", "Yellow", "Black")
    result = wait_for_event(timeout=10)
    clear_output()
    ans9 = result['description']
    data_dict['Question9'].append(ans9)
    if result['description'] == "Yellow":
        points = points + 1
    progress_bar.value +=1
    display(progress_bar)
    count +=1
    print(f'Answered: {count}/20')

    print("Question 10: Which shape has the color green?")
    time.sleep(1)
    answer_options("Lightning", "Moon", "Hexagon", "Square")
    result = wait_for_event(timeout=10)
    clear_output()
    ans10 = result['description']
    data_dict['Question10'].append(ans10)
    if result['description'] == "Square":
        points = points + 1
    print("Next challenge")
    time.sleep(2)
    progress_bar.value +=1
    count +=1
    clear_output()
    return points

In [13]:
def third_set_questions():
    """
    Presents the third set of questions.

    Parameters:
    - None

    Returns:
    - points: Total points earned in the third set of questions.
    """

    global points, count, progress_bar
    display(progress_bar)
    print(f'Answered: {count}/20')
    time.sleep(1)
    print("Answer the following questions")
    time.sleep(1)

    print("Question 11: What shape is inside the pentagon?")
    time.sleep(1)
    answer_options("Pentagon", "Heart", "Smiley Face", "Circle")
    result = wait_for_event(timeout=10)
    clear_output()
    ans11 = result['description']
    data_dict['Question11'].append(ans11)
    if result['description'] == "Heart":
        points = points + 1
    progress_bar.value +=1
    display(progress_bar)
    count +=1
    print(f'Answered: {count}/20')

    print("Question 12: What color is the lightning?")
    time.sleep(1)
    answer_options("Blue", "Red", "Green", "Grey")
    result = wait_for_event(timeout=10)
    clear_output()
    ans12 = result['description']
    data_dict['Question12'].append(ans12)
    if result['description'] == "Green":
        points = points + 1
    progress_bar.value +=1
    display(progress_bar)
    count +=1
    print(f'Answered: {count}/20')

    print("Question 13: Which way is the arrow inside the diamond is facing?")
    time.sleep(1)
    answer_options("Right", "Down", "Up", "Left")
    result = wait_for_event(timeout=10)
    clear_output()
    ans13 = result['description']
    data_dict['Question13'].append(ans13)
    if result['description'] == "Left":
        points = points + 1
    progress_bar.value +=1
    display(progress_bar)
    count +=1
    print(f'Answered: {count}/20')

    print("Question 14: How many smiley faces are there in the image?")
    time.sleep(1)
    answer_options("1", "4", "0", "2")
    result = wait_for_event(timeout=10)
    clear_output()
    ans14 = result['description']
    data_dict['Question14'].append(ans14)
    if result['description'] == "1":
        points = points + 1
    progress_bar.value +=1
    display(progress_bar)
    count +=1
    print(f'Answered: {count}/20')

    print("Question 15: What color is the triangle?")
    time.sleep(1)
    answer_options("Cyan", "Red", "Pink", "Black")
    result = wait_for_event(timeout=10)
    clear_output()
    ans15 = result['description']
    data_dict['Question15'].append(ans15)
    if result['description']== "Pink":
        points = points + 1
    print("Next Challenge")
    time.sleep(2)
    progress_bar.value +=1
    count +=1
    clear_output()
    return points

In [14]:
def fourth_set_questions():
    """
    Presents the fourth set of questions.

    Parameters:
    - None

    Returns:
    - points: Total points earned in the fourth set of questions.
    """

    global points, count, progress_bar
    display(progress_bar)
    print(f'Answered: {count}/20')
    time.sleep(1)
    print("Answer the following questions")
    time.sleep(1)

    print("Question 16: What number is inside the diamond?")
    time.sleep(1)
    answer_options("11", "7", "0", "1")
    result = wait_for_event(timeout=10)
    clear_output()
    ans16 = result['description']
    data_dict['Question16'].append(ans16)
    if result['description'] == "7":
        points = points + 1
    progress_bar.value +=1
    display(progress_bar)
    count +=1
    print(f'Answered: {count}/20')

    print("Question 17: What color is the number inside the square has?")
    time.sleep(1)
    answer_options("Blue", "Green", "Brown", "Yellow")
    result = wait_for_event(timeout=10)
    clear_output()
    ans17 = result['description']
    data_dict['Question17'].append(ans17)
    if result['description'] == "Yellow":
        points = points + 1
    progress_bar.value +=1
    display(progress_bar)
    count +=1
    print(f'Answered: {count}/20')

    print("Question 18: Which number is inside the heart?")
    time.sleep(1)
    answer_options("1", "11", "10", "9")
    result = wait_for_event(timeout=10)
    clear_output()
    ans18 = result['description']
    data_dict['Question18'].append(ans18)
    if result['description'] == "10":
        points = points + 1
    progress_bar.value +=1
    display(progress_bar)
    count +=1
    print(f'Answered: {count}/20')

    print("Question 19: What color does the number 5 has? ")
    time.sleep(1)
    answer_options("Orange", "White", "Black", "Green")
    result = wait_for_event(timeout=10)
    clear_output()
    ans19 = result['description']
    data_dict['Question19'].append(ans19)
    if result['description'] == "Orange":
        points = points + 1
    progress_bar.value +=1
    display(progress_bar)
    count +=1
    print(f'Answered: {count}/20')

    print("Question 20: What color is the star?")
    time.sleep(1)
    answer_options("Orange", "Red", "Black", "Green")
    result = wait_for_event(timeout=10)
    clear_output()
    ans20 = result['description']
    data_dict['Question20'].append(ans20)
    if result['description']== "Black":
        points = points + 1
    progress_bar.value +=1
    count +=1
    clear_output()
    return points

In [18]:
def run_full_test():
    """
    Runs the full memory test, including displaying images and asking questions.

    Returns:
    - None
    """
    data_consent()
    clear_output()
    data_dictionary()  #Create a new dictionary 
    the_starting_page()
    start_time = time.time() #Initialize the timer
    the_images("Sample1.png")
    first_set_questions("Sample2.png")
    the_images("Sample3.png")
    second_set_questions("Sample4.png")
    the_images("Sample5.png")
    third_set_questions()
    the_images("Sample6.png")
    fourth_set_questions()
    end_time = time.time() # End timer 
    time_taken = end_time - start_time # Calculate the time taken
    data_dict['points'].append(points)
    data_dict['time_taken'].append(time_taken)
    print("That's the end of the test")
    time.sleep(1)
    print(f'Your score is {points} out of 20') #Display total score
    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSfRIBk8P49k9Tuti608COVe3OR0u33PaPQM_N5m_taISUFYLw/viewform?usp=sf_link'
    send_to_google_form(data_dict, form_url) 
    time.sleep(1)
    data_dict.clear()  #Delete the elements inside the dictionary
    return

In [19]:
run_full_test()

That's the end of the test
Your score is 17 out of 20
